# MCAV Carla Tutorial

## 1. Background
### 1.1. Acknowledgement
- The base code of this tutorial is kindly open-sourced by Mr Vadium 7s. 
- Please visit his github: https://github.com/vadim7s/SelfDrive/tree/master and his Youtube Channel: https://www.youtube.com/@carlasimulator8782 
### 1.2. Carla Documentation
- Carla Official Documentation: https://carla.readthedocs.io/en/latest/python_api/
- For any further questions please contact: yide.tao@monash.edu

In [1]:
#all imports
import carla #the sim library itself
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping

## 2. Set up the World

In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000)

### 2.1. Reset the World

- The full map name indicate the full path to the map: 'Carla/Maps/Town10HD_Opt'
- While the load_world function only takes in Map Name like "Town10HD_Opt"

In [3]:
# reset the world
full_map_name = client.get_world().get_map().name
current_map = full_map_name.split("/")[-1]
world = client.load_world(current_map)

- `spawn_points` is a list of [`carla.Transform`](https://carla.readthedocs.io/en/0.9.8/python_api/#carla.Transform) objects.

- To retrieve the `(x, y, z)` coordinates of a spawn point, access the `.location` attribute of the `Transform` object.  
  For example, for the first spawn point:

  ```python
  (spawn_points[0].location.x, 
   spawn_points[0].location.y, 
   spawn_points[0].location.z)
    ```

- These spawnpoints are automatically defined by the OpenDRIVE Files associated with the CARLA World.

In [4]:
#define environment/world and get possible places to spawn a car
spawn_points = world.get_map().get_spawn_points()

In [5]:
# print spawn point
def print_spawn_point_xyz(spawn_point):
    """
    Prints the x, y, z coordinates of a given carla.Transform spawn point.

    Parameters:
    spawn_point (carla.Transform): A transform object with a location attribute.
    """
    x = spawn_point.location.x
    y = spawn_point.location.y
    z = spawn_point.location.z
    print(f"x: {x:.2f}, y: {y:.2f}, z: {z:.2f}")

print_spawn_point_xyz(spawn_points[0])

x: 106.42, y: -12.71, z: 0.60


## 3. Vehicle and Camera

### 3.1. Generate Vehicle
- Different vehicles can be found here: https://carla.readthedocs.io/en/latest/catalogue_vehicles/

In [6]:
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

In [7]:
#spawn a car in a random location
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

- Available Transformation:
```python 
# Define the Position and Orientation
position = carla.Location(x=-48.7, y=24.8, z=1.7)
orientation = carla.Rotation(pitch=-13.4, yaw=-75.7, roll=0.0)
transformation = carla.Transform(position, orientation)

# Set Transformation
x.set_transform(transformation)
```

In [8]:
# move simulator view to the car
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
spectator.set_transform(start_point) #move the camera to the vehicle location

- Carla Traffic Manager offers a set of pre-built commands for managing traffics and vehicle motion: https://carla.readthedocs.io/en/latest/adv_traffic_manager/

In [9]:
#send the car off on autopilot - this will leave the spectator
vehicle.set_autopilot(True)

### 3.2. Sensors and Blue Prints

- Carla Blueprint Library offers a set of complete props: https://carla.readthedocs.io/en/latest/bp_library/, which in general can be categorised into:
    1. controller
    2. sensor
    3. static
    4. vehicle
    5. walker

- Carla Actor Library offers a set of convinient tools: https://carla.readthedocs.io/en/latest/core_actors/?utm_source=chatgpt.com
    - The attachment can be attach_to any Carla Object and the AttachmentType can also be varied. 

- The code below will generate a new Actor: "Camera" which follows vehicle around. 

In [ ]:
#setting RGB Camera - this follow the approach explained in a Carla video
# link: https://www.youtube.com/watch?v=om8klsBj4rc&t=1184s

#camera mount offset on the car - you can tweak these to each car to avoid any parts of the car being in the view
CAMERA_POS_Z = 1.6 #this means 1.6m up from the ground
CAMERA_POS_X = 0.9 #this is 0.9m forward

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    # data_dict is used for image access outside of the image. 
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))
    # In the callback function, you can also save image to disk by:
    # data_dict['image'].save_to_disk('out/%06d.png' % image.frame)

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

- Visualize the live-stream video of the actor

In [11]:
while True:
      
    # Dispaly with imshow
    cv2.imshow('All cameras',camera_data['image'])
    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
#grab a snapshot from the camera an show in a pop-up window
img = camera_data['image']
cv2.imshow('RGB Camera',img)
cv2.destroyAllWindows()  # Close all OpenCV windows

In [13]:
# clean up after yourself
camera.stop() # this is the opposite of camera.listen
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

## 4. Additional Sensors

Source: Using the official video: https://www.youtube.com/watch?v=om8klsBj4rc&t=1184s

Available Sensors in Carla: 
1. Cameras
    - RGB, Semantic, Optical Flow, Monocular Depth, Dynamic Vision
    ```python 
    # Normal camera. 
    def rgb_callback(image,data_dict):
        data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

    # Semantic Segmentation camera
    def sem_callback(image,data_dict):
        image.convert(carla.ColorConvert.CityScapesPalette)
        data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

    # Instance Segmentation camera
    def inst_callback(image,data_dict):
        data_dict['inst_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

    # Depth Camera
    def depth_callback(image,data_dict):
        image.convert(carla.ColorConvert.LogarithmicDepth)
        data_dict['depth_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

    # Optical Flow
    def opt_callback(data, data_dict):
        image = data.get_color_coded_flow()
        img = np.reshape(np.copy(image,raw_data), (image.height, image.width, 4))
        img[:, :, 3] = 255
        data_dict['opt_image'] = img
    
    # Dynamic Vision
    def dvs_callback(data, data_dict):
        dvs_events = np.frombuffer(data.raw_data, dtype = np.dtype([
            ('x', np.uint16), ('y', np.uint16), ('t', np.uint64), ('pol', np.bool)
        ]))
        data_dict['dvs_image'] = np.zeros((data.height, data.width, 4), dtype=np.uint8)
        dvs_img = np.zeros((data.height, data.width, 3), dtype = np.uint8)
        dvs_img[dvs_events[:]['y'], dvs_events[:]['x'], dvs_events[:]['pol']*2]=255
        data_dict['dvs_image'][:, :, 0:3] = dvs_image
    ```
2. LIDAR
    - LiDAR visualization can be done using Open3D
3. RADAR
4. Inertial Measurement
5. Collision/obstacle detection (Event Based Sensor)
6. Lane Invasion (Event Based Sensor)
7. Satellite Location